In [7]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
# jupyters notebook Befehl zum direkten Anzeigen von Matplotlib Diagrammen
plt.rcParams['figure.figsize'] = (9, 6)
SMALL_SIZE = 15
MEDIUM_SIZE = 20
BIGGER_SIZE = 25
colormap={0:'red',1:'green'}
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
plt.rcParams['lines.linewidth'] = 1.5
plt.rcParams['axes.linewidth'] = 1.2
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
plt.rcParams['xtick.top'] = True
plt.rcParams['ytick.right'] = True 
plt.rcParams['xtick.labelsize'] = plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['xtick.major.size'] = plt.rcParams['ytick.major.size'] = 7
plt.rcParams['xtick.minor.size'] = plt.rcParams['ytick.minor.size'] = 4
plt.rcParams['xtick.major.width'] = plt.rcParams['ytick.major.width'] = 1.6
plt.rcParams['font.size'] = 12

#loading observations
from observation import load_observations

In [9]:
lam_obs,flux_obs,sig_obs,name_array,e_bv,R_V=load_observations('Example_observation',data_file='SEDobs.dat')

E(B-V)=0.22748
R_V=3.1
Number of datapoints: 80


In [10]:
def chi_window(Fmod,Fphot,sphot,lphot):
    #--------- compute chi in spectral windows --------
    window = [0.3,1.0,3.0,8.0,15.0,30.0,100.0,300.0,1000.0,3000.0,1.E+99]
    iwin = 0
    Nchi = 0
    chi2 = 0.0
    Nwin = 0
    chi2win = 0.0 
    Ndat=len(Fphot)
    for i in range(0,Ndat):
        dchi =  np.log(Fmod[i]/Fphot[i])/(sphot[i]/Fphot[i]) #is this really log or log10
        if lphot[i]>window[iwin]:     # window finished
            if iwin>0:
                print('%3d %3d %7.3f' % (iwin+1,Nwin,np.sqrt(chi2win/Nwin)))
                chi2 += chi2win/Nwin 
                Nchi += 1
            else:
                print()

            iwin += 1
            Nwin = 0
            chi2win = 0.0

        chi2win += dchi**2            # add dchi to window-chi
        Nwin += 1
        print('%13.5f %10.6e %10.6e %6.2f%c %7.2f' % (lphot[i],Fphot[i],Fmod[i],sphot[i]/Fphot[i]*100.0,"%",dchi))

    print(Nchi+1,Nwin,np.sqrt(chi2win/Nwin))
    if Nwin>0:
        chi2 += chi2win/Nwin          # last window
        Nchi += 1

    chi = np.sqrt(chi2/Nchi)        # total chi
    print(' ==>  chi = %8.4f' % (chi))
    return chi


In [11]:
fake_model=flux_obs*1.1
chi_window(fake_model,flux_obs,sig_obs,lam_obs)

      0.15300 1.120793e-12 1.232872e-12   4.72%    2.02
      0.23100 2.699430e-12 2.969373e-12   1.44%    6.61

      0.35160 3.504400e-11 3.854840e-11   1.00%    9.53
      0.35200 1.780018e-11 1.958019e-11   0.48%   19.92
      0.44340 9.600931e-11 1.056102e-10  30.00%    0.32
      0.44400 8.980270e-11 9.878297e-11  42.11%    0.23
      0.44400 9.655478e-11 1.062103e-10   8.39%    1.14
      0.44400 1.033069e-10 1.136375e-10   9.15%    1.04
      0.46770 1.153788e-10 1.269166e-10   1.00%    9.53
      0.48200 1.387007e-10 1.525707e-10   6.28%    1.52
      0.54930 2.308615e-10 2.539476e-10  18.13%    0.53
      0.55400 2.251149e-10 2.476264e-10  18.27%    0.52
      0.55400 2.310675e-10 2.541742e-10   1.87%    5.09
      0.55400 2.310675e-10 2.541742e-10   1.64%    5.81
      0.61680 3.353709e-10 3.689080e-10   1.00%    9.53
      0.62300 4.056582e-10 4.462240e-10   0.71%   13.39
      0.62500 3.031501e-10 3.334651e-10   0.16%   60.24
      0.62500 3.472796e-10 3.820075e-10   2.76%

6.855975540491967

In [24]:
def chi_squared(Fmod,Fphot,sphot,lphot):
    #--------- compute chi properly --------

    Ndat=len(Fphot)
    chi2=0.0
    for i in range(0,Ndat):
        diff=Fphot[i]-Fmod[i]
        chi2+=(diff/sphot[i])**2

    chi = np.sqrt(chi2)        # total chi

    print(' ==>  chi^2 = %8.4f' % (chi2))
    print(' ==>  chi = %8.4f' % (chi))
    return chi2


In [25]:
fake_model=flux_obs*1.1
chi_squared(fake_model,flux_obs,sig_obs,lam_obs)

 ==>  chi^2 = 9207.9878
 ==>  chi =  95.9583


9207.98775579136

In [26]:
def chi_squared_reduced(Fmod,Fphot,sphot,lphot):
    #--------- compute chi properly --------

    Ndat=len(Fphot)
    chi2=0.0
    for i in range(0,Ndat):
        diff=Fphot[i]-Fmod[i]
        chi2+=(diff/sphot[i])**2
    chi2=chi2/Ndat
    chi = np.sqrt(chi2)        # total chi

    print(' ==>  chi^2 = %8.4f' % (chi2))
    print(' ==>  chi = %8.4f' % (chi))
    return chi2


In [27]:
fake_model=flux_obs*1.1
chi_squared_reduced(fake_model,flux_obs,sig_obs,lam_obs)

 ==>  chi^2 = 115.0998
 ==>  chi =  10.7285


115.09984694739201